#### Import Necessary Libraries

In [2]:
import os
import numpy
import pandas as pd
import json
import datetime
import certifi
import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [3]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working

In [5]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "MintChipIcecream54$"

src_dbname = "sakila"
dst_dbname = "sakila_2"

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context

In [7]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

#### Define Functions for Getting Data From and Setting Data Into SQL and Mongo Databases

In [9]:
mysql_args = {
    "uid" : "root",
    "pwd" : "MintChipIcecream54$",
    "hostname" : "localhost",
    "dbname" : "sakila_2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "hqr7gc",
    "password" : "6TeV2LvCQhp246ZJ",
    "cluster_name" : "AlexLaplace",
    "cluster_subnet" : "v7k8y",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila"
}

In [10]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data

In [12]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'JSON files')

json_files = {"film" : 'sakila.film.json',
              "inventory" : 'sakila.inventory.json',
              "store" : 'sakila.store.json',
              "rental" : 'sakila.rental.json',
              "customer" : 'sakila.customer.json',
              "payment" : 'sakila.payment.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)       

In [13]:
data_dir = os.path.join(os.getcwd(), 'JSON files')
print(data_dir)

C:\Users\alexl\DS Systems\Mid-term Project\JSON files


#### Load and Make Transformations to Customer Table from CSV

In [15]:
data_dir = os.path.join(os.getcwd(), 'CSV files')
data_file = os.path.join(data_dir, 'sakila.customer.csv')

df_customer = pd.read_csv(data_file, header=0, index_col=0)
df_customer.head()

,store_id,first_name,last_name,email,address_id,active,create_date,last_update
customer_id,,,,,,,,
1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [16]:
# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customer.insert(0, "customer_key", range(1, df_customer.shape[0]+1))

In [17]:
df_customer.reset_index()

,customer_id,customer_key,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20
...,...,...,...,...,...,...,...,...,...,...
594,595,595,1,TERRENCE,GUNDERSON,TERRENCE.GUNDERSON@sakilacustomer.org,601,1,2006-02-14 22:04:37,2006-02-15 04:57:20
595,596,596,1,ENRIQUE,FORSYTHE,ENRIQUE.FORSYTHE@sakilacustomer.org,602,1,2006-02-14 22:04:37,2006-02-15 04:57:20
596,597,597,1,FREDDIE,DUGGAN,FREDDIE.DUGGAN@sakilacustomer.org,603,1,2006-02-14 22:04:37,2006-02-15 04:57:20
597,598,598,1,WADE,DELVALLE,WADE.DELVALLE@sakilacustomer.org,604,1,2006-02-14 22:04:37,2006-02-15 04:57:20


In [18]:
df_customer = df_customer.reset_index()

In [19]:
df_customer.head()

,customer_id,customer_key,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


#### Read Film Table from new MongoDB Collection "sakila" into Pandas DF

In [21]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "film"

df_film = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [22]:
# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_film.insert(0, "film_key", range(1, df_film.shape[0]+1))
df_film.head(2)

,film_key,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [23]:
df_film.drop(['original_language_id'], axis=1, inplace=True)
df_film.head(2)

,film_key,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [24]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "inventory"

df_inventory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_inventory.head(2)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


In [25]:
# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_inventory.rename(columns={"id":"inventory_id"}, inplace=True)
df_inventory.insert(0, "inventory_key", range(1, df_inventory.shape[0]+1))
df_inventory.head(2)

,inventory_key,inventory_id,film_id,store_id,last_update
0,1,1,1,1,2006-02-15 05:09:17
1,2,2,1,1,2006-02-15 05:09:17


In [26]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "rental"

df_rental = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


In [27]:
# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_rental.rename(columns={"id":"rental_id"}, inplace=True)
df_rental.insert(0, "rental_key", range(1, df_rental.shape[0]+1))
df_rental.head(2)

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


sql_language = "SELECT * FROM sakila.language;"
df_language = get_sql_dataframe(sql_language, mysql_args)
df_language.head(6)

 1. Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['last_update']
df_film.drop(drop_cols, axis=1, inplace=True)

 2. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_film.insert(0, "film_key", range(1, df_film.shape[0]+1))

 3. Display the first 2 rows of the dataframe to validate your work
df_film.head(2)

df_film2 = pd.merge(df_film, df_language, on="language_id")
column_to_move = df_film2.pop("name")
df_film2.insert(6, "language", column_to_move)
df_film2.head(2)

 2. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_actor.insert(0, "actor_key", range(1, df_actor.shape[0]+1))
df_actor.head(2)

df_film2.head(2)

 2. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_store.insert(0, "store_key", range(1, df_store.shape[0]+1))
df_store.head(2)

sql_actor = "SELECT * FROM sakila.actor;"
df_actor = get_dataframe(user_id, pwd, host_name, src_dbname, sql_actor)
df_actor.head(2)

df_actor.drop(['last_update'], axis=1, inplace=True)
df_actor.columns

#### Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

In [37]:
dataframe = df_customer
table_name = 'dim_customer'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [38]:
dataframe = df_film
table_name = 'dim_film'
primary_key = 'film_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [39]:
dataframe = df_inventory
table_name = 'dim_inventory'
primary_key = 'inventory_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [40]:
dataframe = df_rental
table_name = 'dim_rental'
primary_key = 'rental_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### Lookup the Invoice Date Keys from the Date Dimension Table.
#####  Get the Data from the Date Dimension Table.

In [68]:
sql_dim_date = "SELECT date_key, full_date FROM sakila_2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [70]:
df_dim_rental_date = df_dim_date.rename(columns={"date_key" : "rental_return_date_key", "full_date" : "return_date"})
df_rental.return_date = df_rental.return_date.astype('datetime64[ns]').dt.date
df_rental = pd.merge(df_rental, df_dim_rental_date, on='return_date', how='left')
df_rental.drop(['return_date'], axis=1, inplace=True)
df_rental.head(2)

,rental_key,rental_id,rental_date,inventory_id,customer_id,staff_id,last_update,rental_return_date_key
0,1,1,2005-05-24 22:53:30,367,130,1,2006-02-15 21:30:53,20050526
1,2,2,2005-05-24 22:54:33,1525,459,1,2006-02-15 21:30:53,20050528


#### Validate that the New Dimension Tables were Created.

In [72]:
sql_customer = "SELECT * FROM sakila_2.dim_customer;"
df_dim_customer = get_sql_dataframe(sql_customer, **mysql_args)
df_dim_customer.head(2)

,customer_id,customer_key,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [74]:
sql_film = "SELECT * FROM sakila_2.dim_film;"
df_dim_film = get_sql_dataframe(sql_film, **mysql_args)
df_dim_film.head(2)

,film_key,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [76]:
sql_inventory = "SELECT * FROM sakila_2.dim_inventory;"
df_dim_inventory = get_sql_dataframe(sql_inventory, **mysql_args)
df_dim_inventory.head(2)

,inventory_key,inventory_id,film_id,store_id,last_update
0,1,1,1,1,2006-02-15 05:09:17
1,2,2,1,1,2006-02-15 05:09:17


In [78]:
sql_rental = "SELECT * FROM sakila_2.dim_rental;"
df_dim_rental = get_sql_dataframe(sql_rental, **mysql_args)
df_dim_rental.head(2)

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


In [80]:
sql_date = "SELECT * FROM sakila_2.dim_date;"
df_dim_date = get_sql_dataframe(sql_date, **mysql_args)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


### Reading new CVS files

In [83]:
data_dir = os.path.join(os.getcwd(), 'CSV files')
data_file = os.path.join(data_dir, 'sakila.film_category.csv')

df_film_category = pd.read_csv(data_file, header=0, index_col=0)
df_film_category.head()

,category_id,last_update
film_id,,
1,6,2006-02-15 05:07:09
2,11,2006-02-15 05:07:09
3,6,2006-02-15 05:07:09
4,11,2006-02-15 05:07:09
5,8,2006-02-15 05:07:09


In [85]:
data_dir = os.path.join(os.getcwd(), 'CSV files')
data_file = os.path.join(data_dir, 'sakila.category.csv')

df_category = pd.read_csv(data_file, header=0, index_col=0)
df_category.head()

,name,last_update
category_id,,
1,Action,2006-02-15 04:46:27
2,Animation,2006-02-15 04:46:27
3,Children,2006-02-15 04:46:27
4,Classics,2006-02-15 04:46:27
5,Comedy,2006-02-15 04:46:27


In [87]:
df_film_category = df_film_category.reset_index()
df_film_and_category = pd.merge(df_film_category, df_category, on='category_id', how='inner')
df_film_and_category = df_film_and_category.drop('last_update_x', axis=1)
df_film_and_category = df_film_and_category.drop('last_update_y', axis=1)
df_film_and_category['film_key'] = df_film_and_category['film_id']
df_film_and_category.head()

,film_id,category_id,name,film_key
0,1,6,Documentary,1
1,2,11,Horror,2
2,3,6,Documentary,3
3,4,11,Horror,4
4,5,8,Family,5


In [89]:
df_film = pd.merge(df_film, df_film_and_category, on='film_key', how='left')
columns_to_drop = [col for col in df_film.columns if col.endswith('_x') or col.endswith('_y')]
df_film.drop(columns=columns_to_drop, inplace=True)
df_film = df_film.rename(columns={"name":"category_name"})
df_film['film_id'] = df_film['film_key']
df_film.head(2)

,film_key,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,category_id,category_name,film_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary,1
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11,Horror,2


In [91]:
df_inventory = pd.merge(df_inventory, df_film, on='film_id', how='left')
columns_to_drop = [col for col in df_film.columns if col.endswith('_x') or col.endswith('_y')]
df_inventory.drop(columns=columns_to_drop, inplace=True)

df_inventory.head()

,inventory_key,inventory_id,film_id,store_id,last_update_x,film_key,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_y,category_id,category_name
0,1,1,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary
1,2,2,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary
2,3,3,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary
3,4,4,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary
4,5,5,1,2,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary


#### Write Film and Inventory Tables to SQL

In [94]:
dataframe = df_inventory
table_name = 'dim_inventory'
primary_key = 'inventory_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [96]:
dataframe = df_film
table_name = 'dim_film'
primary_key = 'film_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### Validate Tables 

In [98]:
sql_film = "SELECT * FROM sakila_2.dim_film;"
df_dim_film = get_sql_dataframe(sql_film, **mysql_args)
df_dim_film.head(2)

,film_key,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,category_id,category_name,film_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary,1
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11,Horror,2


In [100]:
sql_inventory = "SELECT * FROM sakila_2.dim_inventory;"
df_dim_inventory = get_sql_dataframe(sql_inventory, **mysql_args)
df_dim_inventory.head(2)

,inventory_key,inventory_id,film_id,store_id,last_update_x,film_key,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_y,category_id,category_name
0,1,1,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary
1,2,2,1,1,2006-02-15 05:09:17,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6,Documentary


### Create and Populate the New Fact Tables
#### Extract Data from the Source MongoDB Collections Into DataFrames

In [102]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "rental"

df_fact_rental = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_fact_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


#### Lookup the DateKeys from the Date Dimension Table.

In [104]:
df_dim_rental_date = df_dim_date.rename(columns={"date_key" : "rental_date_key", "full_date" : "rental_date"})
df_fact_rental.rental_date = df_fact_rental.rental_date.astype('datetime64[ns]').dt.date
df_fact_rental = pd.merge(df_fact_rental, df_dim_rental_date, on='rental_date', how='left')
df_fact_rental.drop(['rental_date'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,return_date,staff_id,last_update,rental_date_key,date_name,date_name_us,date_name_eu,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,20050524,2005/05/24,05/24/2005,24/05/2005,...,N,2,2005,2005-05,2005Q2,11,4,2005,2005-11,2005Q4
1,2,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,20050524,2005/05/24,05/24/2005,24/05/2005,...,N,2,2005,2005-05,2005Q2,11,4,2005,2005-11,2005Q4


In [106]:
df_fact_rental.drop(['date_name','date_name_us','date_name_eu','is_last_day_of_month','calendar_quarter','calendar_year','calendar_year_month','calendar_year_qtr','fiscal_month_of_year','fiscal_quarter','fiscal_year','fiscal_year_month','day_of_week','day_of_month','day_of_year','weekday_weekend','week_of_year','month_name','month_of_year','fiscal_year_qtr','day_name_of_week'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,return_date,staff_id,last_update,rental_date_key
0,1,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,20050524
1,2,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,20050524


In [108]:
df_dim_return_date = df_dim_date.rename(columns={"date_key" : "return_date_key", "full_date" : "return_date"})
df_fact_rental.return_date = df_fact_rental.return_date.astype('datetime64[ns]').dt.date
df_fact_rental = pd.merge(df_fact_rental, df_dim_return_date, on='return_date', how='left')
df_fact_rental.drop(['return_date'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,staff_id,last_update,rental_date_key,return_date_key,date_name,date_name_us,date_name_eu,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,367,130,1,2006-02-15 21:30:53,20050524,20050526,2005/05/26,05/26/2005,26/05/2005,...,N,2,2005,2005-05,2005Q2,11,4,2005,2005-11,2005Q4
1,2,1525,459,1,2006-02-15 21:30:53,20050524,20050528,2005/05/28,05/28/2005,28/05/2005,...,N,2,2005,2005-05,2005Q2,11,4,2005,2005-11,2005Q4


In [110]:
df_fact_rental.drop(['date_name','date_name_us','date_name_eu','is_last_day_of_month','calendar_quarter','calendar_year','calendar_year_month','calendar_year_qtr','fiscal_month_of_year','fiscal_quarter','fiscal_year','fiscal_year_month','day_of_week','day_of_month','day_of_year','weekday_weekend','week_of_year','month_name','month_of_year','fiscal_year_qtr','day_name_of_week'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,staff_id,last_update,rental_date_key,return_date_key
0,1,367,130,1,2006-02-15 21:30:53,20050524,20050526
1,2,1525,459,1,2006-02-15 21:30:53,20050524,20050528


In [112]:
df_dim_update_date = df_dim_date.rename(columns={"date_key" : "last_update_key", "full_date" : "last_update"})
df_fact_rental.last_update = df_fact_rental.last_update.astype('datetime64[ns]').dt.date
df_fact_rental = pd.merge(df_fact_rental, df_dim_update_date, on='last_update', how='left')
df_fact_rental.drop(['last_update'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,staff_id,rental_date_key,return_date_key,last_update_key,date_name,date_name_us,date_name_eu,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,367,130,1,20050524,20050526,20060215,2006/02/15,02/15/2006,15/02/2006,...,N,1,2006,2006-02,2006Q1,8,3,2006,2006-08,2006Q3
1,2,1525,459,1,20050524,20050528,20060215,2006/02/15,02/15/2006,15/02/2006,...,N,1,2006,2006-02,2006Q1,8,3,2006,2006-08,2006Q3


In [114]:
df_fact_rental.drop(['date_name','date_name_us','date_name_eu','is_last_day_of_month','calendar_quarter','calendar_year','calendar_year_month','calendar_year_qtr','fiscal_month_of_year','fiscal_quarter','fiscal_year','fiscal_year_month','day_of_week','day_of_month','day_of_year','weekday_weekend','week_of_year','month_name','month_of_year','fiscal_year_qtr','day_name_of_week'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,inventory_id,customer_id,staff_id,rental_date_key,return_date_key,last_update_key
0,1,367,130,1,20050524,20050526,20060215
1,2,1525,459,1,20050524,20050528,20060215


#### Lookup the Primary Keys from the Dimension Tables
**Foreign key relationships** must be established between each newly-crafted **Fact table** and each related **Dimension table**.

#####  First, fetch the Surrogate Primary Key and the Business Key from each Dimension table.

In [116]:
sql_dim_film = "SELECT film_key, film_id FROM sakila_2.dim_film;"
df_dim_film = get_sql_dataframe(sql_dim_film, **mysql_args)
df_dim_film.head(2)

,film_key,film_id
0,1,1
1,2,2


In [118]:
sql_dim_inventory = "SELECT inventory_key, inventory_id, film_id FROM sakila_2.dim_inventory;"
df_dim_inventory = get_sql_dataframe(sql_dim_inventory, **mysql_args)
df_dim_inventory.head(2)

,inventory_key,inventory_id,film_id
0,1,1,1
1,2,2,1


In [120]:
sql_dim_customer = "SELECT customer_key, customer_id FROM sakila_2.dim_customer;"
df_dim_customer = get_sql_dataframe(sql_dim_customer, **mysql_args)
df_dim_customer.head(2)

,customer_key,customer_id
0,1,1
1,2,2


 #### Next, use Business Keys to lookup corresponding Surrogate Primary Keys in the Dimension tables

In [122]:
df_fact_rental = pd.merge(df_fact_rental, df_dim_inventory, on='inventory_id', how='inner')
df_fact_rental.drop(['inventory_id'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_id,customer_id,staff_id,rental_date_key,return_date_key,last_update_key,inventory_key,film_id
0,1,130,1,20050524,20050526,20060215,367,80
1,16,316,2,20050525,20050526,20060215,389,86


In [124]:
df_fact_rental.insert(0, "fact_rental_key", range(1, df_fact_rental.shape[0]+1))
df_fact_rental.head(2)

,fact_rental_key,rental_id,customer_id,staff_id,rental_date_key,return_date_key,last_update_key,inventory_key,film_id
0,1,1,130,1,20050524,20050526,20060215,367,80
1,2,16,316,2,20050525,20050526,20060215,389,86


#### Load fact table back to datawarehouse and validate

In [126]:
dataframe = df_fact_rental
table_name = 'fact_rental'
primary_key = 'fact_rental_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [128]:
sql_fact_rental = "SELECT * FROM sakila_2.fact_rental;"
df_dim_rental = get_sql_dataframe(sql_fact_rental, **mysql_args)
df_dim_rental.head()

,fact_rental_key,rental_id,customer_id,staff_id,rental_date_key,return_date_key,last_update_key,inventory_key,film_id
0,1,1,130,1,20050524,20050526,20060215,367,80
1,2,16,316,2,20050525,20050526,20060215,389,86
2,3,17,575,1,20050525,20050527,20060215,830,181
3,4,21,388,2,20050525,20050526,20060215,146,31
4,5,22,509,2,20050525,20050526,20060215,727,159


#### Prooving Existence of New DW with Queries

In [134]:
sql_purchase_orders = """
    SELECT
        fr.film_id,
        fi.title AS film_title,
        c.customer_id,
        CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
        COUNT(fr.rental_id) AS total_rentals_for_film,
        SUM(COUNT(fr.rental_id)) OVER (PARTITION BY c.customer_id) AS total_rentals_for_customer
    FROM
        fact_rental fr
    JOIN
        dim_inventory i ON fr.inventory_key = i.inventory_key
    JOIN
        dim_film fi ON i.film_id = fi.film_key
    JOIN
        dim_customer c ON fr.customer_id = c.customer_id
    GROUP BY 
        fr.film_id, fi.title, c.customer_id, customer_name
    ORDER BY
        customer_id ASC;
"""

df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders

,film_id,film_title,customer_id,customer_name,total_rentals_for_film,total_rentals_for_customer
0,83,BLUES INSTINCT,6,JENNIFER DAVIS,1,1.0
1,196,CRUELTY UNFORGIVEN,9,MARGARET MOORE,1,1.0
2,86,BOOGIE AMELIE,11,LISA ANDERSON,1,1.0
3,174,CONFIDENTIAL INTERVIEW,18,CAROL GARCIA,1,1.0
4,83,BLUES INSTINCT,19,RUTH MARTINEZ,1,2.0
...,...,...,...,...,...,...
207,1,ACADEMY DINOSAUR,587,SERGIO STANFIELD,1,1.0
208,167,COMA HEAD,589,TRACY HERRMANN,1,2.0
209,179,CONQUERER NUTS,589,TRACY HERRMANN,1,2.0
210,60,BEAST HUNCHBACK,595,TERRENCE GUNDERSON,1,1.0
